# Libraries

In [52]:
import pandas as pd
import math
import numpy as np

# Remove users

In [59]:
files = ['adjacency_matrix_not_directed_48_7_categories_US.csv',
        'adjacency_matrix_weekday_not_directed_48_7_categories_US.csv',
        'adjacency_matrix_weekend_not_directed_48_7_categories_US.csv',
        'distance_matrix_not_directed_48_7_categories_US.csv',
        'duration_matrix_not_directed_48_7_categories_US.csv',
        'features_matrix_not_directed_48_7_categories_US.csv',
        'features_matrix_weekday_not_directed_48_7_categories_US.csv',
        'features_matrix_weekend_not_directed_48_7_categories_US.csv']

limit = 10000000
for file in files:
    df = pd.read_csv('gowalla/'+file)
    df.drop(df.loc[df['user_id'] > limit].index, inplace=True)    
    df.to_csv('gowalla/processed/'+file, index=False)

# Pre-process analysis

In [92]:
gowalla = pd.read_csv('gowalla/gowalla_checkins.csv', nrows=1000)

In [5]:
gowalla.head()

,userid,placeid,datetime
0,1338,482954,2011-06-23T02:24:22Z
1,1338,580963,2011-06-22T14:23:03Z
2,1338,365256,2011-06-09T23:29:30Z
3,1338,89504,2011-05-22T15:54:30Z
4,1338,1267135,2011-05-21T16:51:13Z


In [93]:
gowalla['userid'].value_counts().sum() == len(gowalla)

True

In [6]:
import json

data_json = 'gowalla/gowalla_category_structure.json'

with open(data_json, 'r') as file:
    data = json.load(file)
    
data

{'spot_categories': [{'name': 'Community',
   'image_url': 'http://static.gowalla.com/categories/934-c4bb6b0777a2de0f060c4d71b0db0ed8-100.png',
   'url': '/categories/934',
   'spot_categories': [{'name': 'Campus Spot',
     'image_url': 'http://static.gowalla.com/categories/133-6437b3f0e56f2414baa34781588befd1-100.png',
     'url': '/categories/133',
     'spot_categories': [{'name': 'Administration',
       'image_url': 'http://static.gowalla.com/categories/135-83777de928947312f311f05418a6a6f8-100.png',
       'url': '/categories/135',
       'description': '',
       'small_image_url': 'http://static.gowalla.com/categories/135-b8741a5f83c1189533cf2f0712e4d8e3-30.png'},
      {'name': 'Campus Commons',
       'image_url': 'http://static.gowalla.com/categories/138-2abf3dcbc010dbdba29f827a2264deec-100.png',
       'url': '/categories/138',
       'description': '',
       'small_image_url': 'http://static.gowalla.com/categories/138-36f6e10b93d0edcf483113a95a82f10b-30.png'},
      {'nam

In [94]:
gowalla_friendship = pd.read_csv('gowalla/gowalla_friendship.csv', nrows=1000)
gowalla_friendship.head()

,userid1,userid2
0,1,63488
1,1,2
2,1,3
3,1,4
4,1,5


In [8]:
gowalla_spots_subset1 = pd.read_csv('gowalla/gowalla_spots_subset1.csv', nrows=1000)
gowalla_spots_subset1.head()

,id,created_at,lng,lat,photos_count,checkins_count,users_count,radius_meters,highlights_count,items_count,max_items_count,spot_categories
0,8904,2008-12-06T16:28:53Z,-94.607499,39.052318,0,114,21,35,0,10,10,"[{'url': '/categories/89', 'name': 'Craftsman'}]"
1,8932,2008-12-13T02:16:51Z,-97.254356,32.927662,2,67,48,75,0,6,10,"[{'url': '/categories/17', 'name': 'BBQ'}]"
2,8936,2008-12-14T22:08:39Z,-94.591995,39.053318,0,75,46,75,0,10,10,"[{'url': '/categories/103', 'name': 'Theatre'}]"
3,8938,2008-12-15T00:22:49Z,-94.590311,39.052824,38,438,94,50,10,10,10,"[{'url': '/categories/1', 'name': 'Coffee Shop'}]"
4,8947,2008-12-16T23:14:05Z,-122.029631,37.331880,91,3100,1186,200,20,10,10,"[{'url': '/categories/121', 'name': 'Corporate..."


In [10]:
gowalla_spots_subset2 = pd.read_csv('gowalla/gowalla_spots_subset2.csv', nrows=1000, encoding='latin-1')
gowalla_spots_subset2.head()

,id,lat,lng,name,city_state,Unnamed: 5,Unnamed: 6
0,9813,36.631508,-121.912022,Pacific Ocean,"Seaside, CA",NaN,NaN
1,11981,40.690173,-74.045491,Refreshment Park,"New York, NY",NaN,NaN
2,12455,36.642022,-93.284912,Mickey Gilley Theatre,"Branson, MO",NaN,NaN
3,12455,36.642022,-93.284912,Mickey Gilley Theatre,"Branson, MO",NaN,NaN
4,13810,42.548879,-82.902768,Jaword,"Clinton, MI",NaN,NaN


In [14]:
gowalla_userinfo = pd.read_csv('gowalla/gowalla_userinfo.csv', nrows=1000)
gowalla_userinfo.head()

,id,bookmarked_spots_count,challenge_pin_count,country_pin_count,highlights_count,items_count,photos_count,pins_count,province_pin_count,region_pin_count,state_pin_count,trips_count,friends_count,stamps_count,checkin_num,places_num
0,1,26,59,1,29,18,135,85,0,13,12,2,372,757,1766,756
1,2,36,79,12,117,29,1069,138,0,31,19,7,775,1227,2892,1220
2,3,11,70,1,42,49,149,93,0,7,6,2,100,905,3021,905
3,4,7,51,1,13,29,138,80,0,13,12,1,179,608,1325,613
4,5,34,82,5,62,36,1012,134,0,19,14,2,525,1124,3215,1124


# Generating original dataset

In [3]:
# gowalla_columns = ['user_id', 'time','lat','lon', 'loc_id']
# g_dataset = pd.read_csv('gowalla/loc-gowalla_totalCheckins.txt/Gowalla_totalCheckins.txt', names = gowalla_columns, index_col=False, delimiter = "\t")

In [97]:
g_dataset = pd.read_csv('gowalla/checkins.csv', index_col=False, usecols = ['userid', 'datetime', 'lat', 'lng', 'placeid', 'categoryid', 'NAME'])
g_dataset = g_dataset.dropna()

In [83]:
g_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567352 entries, 0 to 875915
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   userid      567352 non-null  int64  
 1   placeid     567352 non-null  int64  
 2   datetime    567352 non-null  object 
 3   lng         567352 non-null  float64
 4   lat         567352 non-null  float64
 5   categoryid  567352 non-null  int64  
 6   NAME        567352 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 34.6+ MB


In [98]:
g_dataset['country'] = 'United States'

In [99]:
g_dataset = g_dataset.rename(columns={'NAME': 'state'})

In [100]:
# datetime converter
g_dataset['datetime'] = pd.to_datetime(g_dataset['datetime'])
cols = ['userid', 'categoryid', 'placeid', 'datetime', 'lat', 'lng', 'country', 'state']
g_dataset = g_dataset[cols]

In [87]:
g_dataset

,userid,categoryid,placeid,datetime,lat,lng,country,state
0,1338,3,482954,2011-06-23 02:24:22+00:00,44.934198,-93.177506,United States,Minnesota
1,116691,3,482954,2011-06-05 17:42:29+00:00,44.934198,-93.177506,United States,Minnesota
2,59161,3,482954,2011-01-14 01:39:11+00:00,44.934198,-93.177506,United States,Minnesota
3,59161,3,482954,2010-07-23 01:32:22+00:00,44.934198,-93.177506,United States,Minnesota
4,149777,3,482954,2010-03-18 01:40:40+00:00,44.934198,-93.177506,United States,Minnesota
...,...,...,...,...,...,...,...,...
875911,8852,0,16007,2009-10-27 23:39:02+00:00,46.730412,-117.158010,United States,Washington
875912,8852,2,109832,2009-11-18 02:39:03+00:00,46.732904,-117.030310,United States,Idaho
875913,8852,5,109279,2009-11-17 21:55:47+00:00,46.732074,-116.999127,United States,Idaho
875914,8852,2,83395,2009-11-17 07:19:14+00:00,46.732867,-117.038087,United States,Idaho


In [101]:
cols = ['userid', 'category', 'placeid', 'local_datetime', 'latitude', 'longitude', 'country_name', 'state_name']
# Criar um dicionário que mapeia os nomes atuais das colunas para os novos nomes
maps = dict(zip(g_dataset.columns, cols))

# Renomear as colunas usando o método rename()
g_dataset = g_dataset.rename(columns=maps)

In [89]:
g_dataset

,userid,category,placeid,local_datetime,latitude,longitude,country_name,state_name
0,1338,3,482954,2011-06-23 02:24:22+00:00,44.934198,-93.177506,United States,Minnesota
1,116691,3,482954,2011-06-05 17:42:29+00:00,44.934198,-93.177506,United States,Minnesota
2,59161,3,482954,2011-01-14 01:39:11+00:00,44.934198,-93.177506,United States,Minnesota
3,59161,3,482954,2010-07-23 01:32:22+00:00,44.934198,-93.177506,United States,Minnesota
4,149777,3,482954,2010-03-18 01:40:40+00:00,44.934198,-93.177506,United States,Minnesota
...,...,...,...,...,...,...,...,...
875911,8852,0,16007,2009-10-27 23:39:02+00:00,46.730412,-117.158010,United States,Washington
875912,8852,2,109832,2009-11-18 02:39:03+00:00,46.732904,-117.030310,United States,Idaho
875913,8852,5,109279,2009-11-17 21:55:47+00:00,46.732074,-116.999127,United States,Idaho
875914,8852,2,83395,2009-11-17 07:19:14+00:00,46.732867,-117.038087,United States,Idaho


In [102]:
categories = pd.read_csv('processed_data/category_names.csv')

In [91]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        7 non-null      object
 1   categoryid  7 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 240.0+ bytes


In [103]:
g_dataset = pd.merge(g_dataset, categories, left_on="category", right_on="categoryid")

In [105]:
g_dataset['category'] = g_dataset['name']

In [107]:
g_dataset.drop(['name', 'categoryid'], axis=1, inplace=True)

In [108]:
g_dataset

,userid,category,placeid,local_datetime,latitude,longitude,country_name,state_name
0,1338,Nightlife,482954,2011-06-23 02:24:22+00:00,44.934198,-93.177506,United States,Minnesota
1,116691,Nightlife,482954,2011-06-05 17:42:29+00:00,44.934198,-93.177506,United States,Minnesota
2,59161,Nightlife,482954,2011-01-14 01:39:11+00:00,44.934198,-93.177506,United States,Minnesota
3,59161,Nightlife,482954,2010-07-23 01:32:22+00:00,44.934198,-93.177506,United States,Minnesota
4,149777,Nightlife,482954,2010-03-18 01:40:40+00:00,44.934198,-93.177506,United States,Minnesota
...,...,...,...,...,...,...,...,...
567347,57784,Outdoors,862189,2010-05-02 14:18:15+00:00,30.331140,-87.139284,United States,Florida
567348,8852,Outdoors,1006077,2011-02-20 19:56:00+00:00,48.535292,-123.014495,United States,Washington
567349,8852,Outdoors,1576881,2010-08-12 22:16:05+00:00,33.529635,-117.773644,United States,California
567350,8852,Outdoors,1576881,2010-08-11 23:16:16+00:00,33.529635,-117.773644,United States,California


In [109]:
g_dataset = g_dataset.sample(frac=0.3)

In [110]:
g_dataset.to_csv('gowalla/checks.csv', index=False)